In [417]:
import deepcut
import numpy as np
from tensorflow.keras.preprocessing import sequence
from tensorflow import keras
from pythainlp import word_vector
import re,string
from pythainlp import word_vector
from numpy import argmax
import tensorflow as tf
from tensorflow.keras.utils import to_categorical
import random
from keras_contrib.layers import CRF

In [2]:
def clean_msg(msg):
    
    # ลบ text ที่อยู่ในวงเล็บ <> ทั้งหมด
    msg = re.sub(r'<.*?>','', msg)
    msg = re.sub(r'\d+', '',msg)
    # ลบ hashtag
    msg = re.sub(r'#','',msg)
    msg = re.sub(r'-','',msg)
    msg = re.sub(r'ฯ','',msg)
    msg = re.sub(r'ๆ','',msg)
    msg = re.sub(r'!@#$','',msg)
    msg = re.sub(r'[a-zA-Z]','',msg)
    # ลบ เครื่องหมายคำพูด (punctuation)
    for c in string.punctuation:
        msg = re.sub(r'\{}'.format(c),'',msg)
    # ลบ separator เช่น \n \t
    msg = ' '.join(msg.split())
    
    return msg

In [239]:
input_txt = open("corpus1.txt","r")
text = input_txt.read()
txt = clean_msg(text)
input_txt.close()
corpus = txt.split()
print(corpus)

['พยากรณ์อากาศ', 'ชั่วโมงข้างหน้า', 'บริเวณความกดอากาศสูงหรือมวลอากาศเย็นกำลังปานกลางจากประเทศจีนแผ่ปกคลุมประเทศไทยตอนบน', 'ประกอบกับมีหย่อมความกดอากาศต่ำปกคลุมบริเวณประเทศกัมพูชา', 'ลักษณะเช่นนี้ทำให้ประเทศไทยตอนบนมีฝนเกิดขึ้น', 'กับมีอากาศเย็น', 'ส่วนบริเวณยอดดอยและยอดภู', 'มีอากาศหนาว', 'สำหรับมรสุมตะวันออกเฉียงเหนือพัดปกคลุมอ่าวไทยและภาคใต้', 'ทำให้ภาคใต้มีฝนตกต่อเนื่อง', 'อนึ่ง', 'พายุโซนร้อน', 'อัสนี', 'พายุระดับ', 'บริเวณตะวันตกเฉียงใต้ของเกาะไต้หวัน', 'มีแนวโน้มที่จะอ่อนกำลังลง', 'และไม่มีผลกระทบต่อประเทศไทย', 'ในช่วงวันที่', 'พย', 'บริเวณความกดอากาศสูงหรือมวลอากาศเย็นกำลังปานกลางจากประเทศจีนปกคลุมประเทศไทยตอนบน', 'ประกอบกับมีหย่อมความกดอากาศต่ำปกคลุมบริเวณประเทศกัมพูชา', 'ลักษณะเช่นนี้ทำให้บริเวณประเทศไทยตอนบนมีอากาศเย็นกับมีฝนเล็กน้อยถึงปานกลาง', 'โดยเฉพาะบริเวณภาคตะวันออกเฉียงเหนือตอนล่างและภาคตะวันออก', 'สำหรับภาคใต้ตอนล่างมีฝนตกหนักบางแห่ง', 'เนื่องจากมรสุมตะวันออกเฉียงเหนือพัดปกคลุมอ่าวไทยและภาคใต้', 'ส่วนในช่วงวันที่', 'พย', 'บริเวณความกดอากาศสูงหรือมวลอากาศเย็นกำลังค่อน

In [375]:
def word_tokenize(corpus):
    x = 0
    words = []
    for i in corpus:
        list_word = deepcut.tokenize(i,custom_dict='custom_dict.txt') #ตัดคำ
        words.append(list_word)
        x += 1
    return words
# corpus=word_tokenize(corpus)
# x_test = word_tokenize(x_test)
words = word_tokenize(corpus)
print(words)
# print(len(words))

[['พยากรณ์อากาศ'], ['ชั่วโมง', 'ข้าง', 'หน้า'], ['บริเวณ', 'ความ', 'กด', 'อากาศ', 'สูง', 'หรือ', 'มวล', 'อากาศ', 'เย็น', 'กำลัง', 'ปานกลาง', 'จาก', 'ประเทศ', 'จีน', 'แผ่', 'ปกคลุม', 'ประเทศ', 'ไทย', 'ตอน', 'บน'], ['ประกอบ', 'กับ', 'มี', 'หย่อม', 'ความ', 'กด', 'อากาศ', 'ต่ำ', 'ปกคลุม', 'บริเวณ', 'ประเทศ', 'กัมพูชา'], ['ลักษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น'], ['กับ', 'มี', 'อากาศ', 'เย็น'], ['ส่วน', 'บริเวณ', 'ยอด', 'ดอย', 'และ', 'ยอดภู'], ['มี', 'อากาศ', 'หนาว'], ['สำหรับ', 'มรสุม', 'ตะวัน', 'ออก', 'เฉียง', 'เหนือ', 'พัด', 'ปกคลุม', 'อ่าวไทย', 'และ', 'ภาค', 'ใต้'], ['ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่อง'], ['อนึ่ง'], ['พายุ', 'โซนร้อน'], ['อัสนี'], ['พายุ', 'ระดับ'], ['บริเวณ', 'ตะวัน', 'ตก', 'เฉียง', 'ใต้', 'ของ', 'เกาะ', 'ไต้หวัน'], ['มี', 'แนวโน้ม', 'ที่', 'จะ', 'อ่อน', 'กำลัง', 'ลง'], ['และ', 'ไม่', 'มี', 'ผล', 'กระทบ', 'ต่อ', 'ประเทศ', 'ไทย'], ['ใน', 'ช่วง', 'วัน', 'ที่'], ['พย'], ['บริเวณ', 'ความ', 'กด', 'อา

In [376]:
print(words)

[['พยากรณ์อากาศ'], ['ชั่วโมง', 'ข้าง', 'หน้า'], ['บริเวณ', 'ความ', 'กด', 'อากาศ', 'สูง', 'หรือ', 'มวล', 'อากาศ', 'เย็น', 'กำลัง', 'ปานกลาง', 'จาก', 'ประเทศ', 'จีน', 'แผ่', 'ปกคลุม', 'ประเทศ', 'ไทย', 'ตอน', 'บน'], ['ประกอบ', 'กับ', 'มี', 'หย่อม', 'ความ', 'กด', 'อากาศ', 'ต่ำ', 'ปกคลุม', 'บริเวณ', 'ประเทศ', 'กัมพูชา'], ['ลักษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น'], ['กับ', 'มี', 'อากาศ', 'เย็น'], ['ส่วน', 'บริเวณ', 'ยอด', 'ดอย', 'และ', 'ยอดภู'], ['มี', 'อากาศ', 'หนาว'], ['สำหรับ', 'มรสุม', 'ตะวัน', 'ออก', 'เฉียง', 'เหนือ', 'พัด', 'ปกคลุม', 'อ่าวไทย', 'และ', 'ภาค', 'ใต้'], ['ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่อง'], ['อนึ่ง'], ['พายุ', 'โซนร้อน'], ['อัสนี'], ['พายุ', 'ระดับ'], ['บริเวณ', 'ตะวัน', 'ตก', 'เฉียง', 'ใต้', 'ของ', 'เกาะ', 'ไต้หวัน'], ['มี', 'แนวโน้ม', 'ที่', 'จะ', 'อ่อน', 'กำลัง', 'ลง'], ['และ', 'ไม่', 'มี', 'ผล', 'กระทบ', 'ต่อ', 'ประเทศ', 'ไทย'], ['ใน', 'ช่วง', 'วัน', 'ที่'], ['พย'], ['บริเวณ', 'ความ', 'กด', 'อา

In [377]:
thai_letters = 'กขฃคฅฆงจฉชซฌญฎฏฐฑฒณดตถทธนบปผฝพฟภมยรฤฤๅลฦฦๅวศษสหฬอฮะัาำิีึืุูเแโใไ็่้๊๋์'

In [378]:
def ran_incorect(w):
    incorect = []
    for i in w:
        in_cor = []
        for r in i:
            if len(r) >= 5:
                x = random.randint(1, len(r)-1)
                y = random.randint(0, len(thai_letters)-1)
                x1 = []
                for ran in r:
                    x1.append(ran)
                x1[x] = thai_letters[y]
                j = ''.join(x1)
                in_cor.append(j)
            else:
                in_cor.append(r)
        incorect.append(in_cor)
    return incorect
#                 print(r[1])
#             print(len(r))

In [398]:
check = {}
check[' '] = 0
check['corect'] = 1
check['incorect'] = 2

In [380]:
random.seed(1)
incorect_word = ran_incorect(words)
print(incorect_word)
print(len(incorect_word))

[['พยาฉรณ์อากาศ'], ['ชั่ฐโมง', 'ข้าง', 'หน้า'], ['บริเืณ', 'ความ', 'กด', 'อากาอ', 'สูง', 'หรือ', 'มวล', 'อาญาศ', 'เย็น', 'กำลัค', 'ปานกีาง', 'จาก', 'ประเทก', 'จีน', 'แผ่', 'ปกคลรม', 'ปรฎเทศ', 'ไทย', 'ตอน', 'บน'], ['ประคอบ', 'กับ', 'มี', 'หค่อม', 'ความ', 'กด', 'ออกาศ', 'ต่ำ', 'ปกิลุม', 'บ้ิเวณ', 'ปรึเทศ', 'กัมพ์ชา'], ['ลัษษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ปรฝเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น'], ['กับ', 'มี', 'อากาๅ', 'เย็น'], ['ส่วน', 'บำิเวณ', 'ยอด', 'ดอย', 'และ', 'ยธดภู'], ['มี', 'อากฐศ', 'หนาว'], ['สำหไับ', 'มรสุไ', 'ตะลัน', 'ออก', 'เฉีใง', 'เหนืฅ', 'พัด', 'ปกคลภม', 'อ่าวไทั', 'และ', 'ภาค', 'ใต้'], ['ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่ทง'], ['อนึ์ง'], ['พายุ', 'โซนร้อฬ'], ['อึสนี'], ['พายุ', 'รตดับ'], ['บริเวะ', 'ตะวโน', 'ตก', 'เเียง', 'ใต้', 'ของ', 'เกาะ', 'ไฦ้หวัน'], ['มี', 'แนวโน้ะ', 'ที่', 'จะ', 'อ่อน', 'กำถัง', 'ลง'], ['และ', 'ไม่', 'มี', 'ผล', 'กรขทบ', 'ต่อ', 'ปร๋เทศ', 'ไทย'], ['ใน', 'ช่วง', 'วัน', 'ที่'], ['พย'], ['บริเวพ', 'ความ', 'กด', 'อา

In [381]:
def create_tar(word,ch):
    tar_in = []
    for i in word:
        targ = []
        for j in i:
            if ch:
                if len(j) >= 5:
                    targ.append('incorect')
                else:
                    targ.append('corect')
            else:
                targ.append('corect')
        tar_in.append(targ)
    return tar_in
# print(tar_in)

In [382]:
target_c = create_tar(words,False)
target_ic = create_tar(incorect_word,True)

In [383]:
target_c.extend(target_ic)
words.extend(incorect_word)
print(words)

[['พยากรณ์อากาศ'], ['ชั่วโมง', 'ข้าง', 'หน้า'], ['บริเวณ', 'ความ', 'กด', 'อากาศ', 'สูง', 'หรือ', 'มวล', 'อากาศ', 'เย็น', 'กำลัง', 'ปานกลาง', 'จาก', 'ประเทศ', 'จีน', 'แผ่', 'ปกคลุม', 'ประเทศ', 'ไทย', 'ตอน', 'บน'], ['ประกอบ', 'กับ', 'มี', 'หย่อม', 'ความ', 'กด', 'อากาศ', 'ต่ำ', 'ปกคลุม', 'บริเวณ', 'ประเทศ', 'กัมพูชา'], ['ลักษณะ', 'เช่น', 'นี้', 'ทำ', 'ให้', 'ประเทศ', 'ไทย', 'ตอน', 'บน', 'มี', 'ฝน', 'เกิด', 'ขึ้น'], ['กับ', 'มี', 'อากาศ', 'เย็น'], ['ส่วน', 'บริเวณ', 'ยอด', 'ดอย', 'และ', 'ยอดภู'], ['มี', 'อากาศ', 'หนาว'], ['สำหรับ', 'มรสุม', 'ตะวัน', 'ออก', 'เฉียง', 'เหนือ', 'พัด', 'ปกคลุม', 'อ่าวไทย', 'และ', 'ภาค', 'ใต้'], ['ทำ', 'ให้', 'ภาค', 'ใต้', 'มี', 'ฝน', 'ตก', 'ต่อเนื่อง'], ['อนึ่ง'], ['พายุ', 'โซนร้อน'], ['อัสนี'], ['พายุ', 'ระดับ'], ['บริเวณ', 'ตะวัน', 'ตก', 'เฉียง', 'ใต้', 'ของ', 'เกาะ', 'ไต้หวัน'], ['มี', 'แนวโน้ม', 'ที่', 'จะ', 'อ่อน', 'กำลัง', 'ลง'], ['และ', 'ไม่', 'มี', 'ผล', 'กระทบ', 'ต่อ', 'ประเทศ', 'ไทย'], ['ใน', 'ช่วง', 'วัน', 'ที่'], ['พย'], ['บริเวณ', 'ความ', 'กด', 'อา

In [5]:
window_size = 1
def generate_data(corpus, window_size = 2):
    maxlen = window_size*2
    contexts = []
    labels   = []   
    for words in corpus:
        L = len(words)
        if len(words) < 2:
            contexts.append([''])
            labels.append(words[0])
        else:
            for index, word in enumerate(words):

                s = index - window_size
                e = index + window_size + 1

                contexts.append([words[i] for i in range(s, e) if 0 <= i < L and i != index])
                labels.append(word)

    return contexts,labels
    #ฟังก์ชั้นสำหรับแบ่งข้อมูลให้อยู่ในรูป target and label

In [6]:
w2v = word_vector.get_model()
th2v = w2v.get_w

In [282]:
vecter = []
vocab_ = []
index = 0
for word in w2v.index2word:
    vocab_.append(word)
    vecter.append(w2v[word])

In [283]:
max_len = 50
max_len_char = 30

character_LSTM_unit = 32
char_embedding_dim = 32
main_lstm_unit = 256 ## Bidirectional 256 + 256 = 512
lstm_recurrent_dropout = 0.5

train_batch_size = 32
train_epochs = 50

In [284]:
def sequence_word(input_text):
    idxs = list()
    for word in input_text:
        if word in vocab_:
            idxs.append(vocab_.index(word))
        else:
            vocab_.append(word)
            vecter.append(np.random.rand(*vecter[0].shape))
            idxs.append(vocab_.index(word))
    return idxs

def sequence_target(input_label):
    idxs = [ner_to_ix[w] for w in input_label]
    return idxs

In [288]:
X_word_train = [sequence_word(s) for s in words]
X_word_train =sequence.pad_sequences(np.array(X_word_train),maxlen=max_len,padding='pre')

In [289]:
print(X_word_train.shape)

(176, 50)


In [287]:
# vocab_ = [''] + vocab_
# vecter = np.vstack((np.random.rand(*vecter[0].shape), vecter))

In [290]:
char_to_int = dict((c, i) for i, c in enumerate(thai_letters,start=1))
int_to_char = dict((i, c) for i, c in enumerate(thai_letters,start=1))

In [291]:
d = 'พยากรณ์อากาศ'
print(words[0])

['พยากรณ์อากาศ']


In [292]:
thai2dict = {}
for word in w2v.index2word:
    thai2dict[word] = w2v[word]

In [293]:
chars = set([w_i for w in thai2dict for w_i in w])
char2idx = {c: i + 5 for i, c in enumerate(chars)}

char2idx["pad"] = 0
char2idx["unknown"] = 1
char2idx[" "] = 2

char2idx["$"] = 3
char2idx["#"] = 4
char2idx["!"] = 5
char2idx["%"] = 6
char2idx["&"] = 7
char2idx["*"] = 8
char2idx["+"] = 9
char2idx[","] = 10
char2idx["-"] = 11
char2idx["."] = 12
char2idx["/"] = 13
char2idx[":"] = 14
char2idx[";"] = 15
char2idx["?"] = 16
char2idx["@"] = 17
char2idx["^"] = 18
char2idx["_"] = 19
char2idx["`"] = 20
char2idx["="] = 21
char2idx["|"] = 22
char2idx["~"] = 23
char2idx["'"] = 24
char2idx['"'] = 25

char2idx["("] = 26
char2idx[")"] = 27
char2idx["{"] = 28
char2idx["}"] = 29
char2idx["<"] = 30
char2idx[">"] = 31
char2idx["["] = 32
char2idx["]"] = 33

n_chars = len(char2idx)
print(n_chars)
print(char2idx)

404
{'г': 5, '♣': 6, 'ɛ': 7, '・': 8, '<': 30, 'ล': 10, 'ê': 11, '∑': 12, '》': 13, 'ħ': 14, 'π': 15, '년': 16, 'ờ': 17, '9': 18, 'ë': 19, 'เ': 20, 'ç': 21, 'ฬ': 22, '²': 23, '▪': 24, 'ø': 25, '：': 26, '̰': 27, 'ญ': 28, '๒': 29, '일': 30, 'ą': 31, 'f': 32, '์': 33, '′': 34, 'ج': 35, 'é': 36, 'ó': 37, 'ح': 38, 'น': 39, 'w': 40, '±': 41, 'ã': 42, 'ę': 43, 'ก': 44, 'í': 45, '̚': 46, '、': 47, 'ธ': 48, 'ñ': 49, 'ρ': 50, 'z': 51, 'j': 52, 'ː': 53, '๗': 54, 'ṣ': 55, 'º': 56, 'ш': 57, 'พ': 58, 'e': 59, 'è': 60, '!': 5, 'ő': 62, 'r': 63, ')': 27, 'َ': 65, '—': 66, 'ิ': 67, '๋': 68, 'α': 69, 'ề': 70, 'ª': 71, 'ا': 72, 'ō': 73, '−': 74, '.': 12, '\uf712': 76, '’': 77, '★': 78, 'ั': 79, '„': 80, 'ณ': 81, 'ṭ': 82, 'ǎ': 83, '德': 84, 'ộ': 85, 'b': 86, ']': 33, 'ะ': 88, 'å': 89, '\x93': 90, 'ฎ': 91, 'θ': 92, '月': 93, 'ๆ': 94, '³': 95, 'ω': 96, 'ฃ': 97, 'ʰ': 98, 'ọ': 99, ';': 15, 'à': 101, 'ภ': 102, 'g': 103, '\\': 104, 's': 105, 'ถ': 106, '2': 107, 'ŭ': 108, '̍': 109, '\uf70a': 110, 'ʔ': 111, 'ำ': 112, 'δ

In [295]:
X_char_tr = []
for sentence in words:
    sent_seq = []
    for i in range(max_len):
        word_seq = []
        for j in range(max_len_char):
            try:
                if(sentence[i][0][j] in char2idx):
                    word_seq.append(char2idx.get(sentence[i][0][j]))
                else:
                    word_seq.append(char2idx.get("unknown"))
            except:
                word_seq.append(char2idx.get("pad"))
        sent_seq.append(word_seq)
    X_char_tr.append(np.array(sent_seq))

In [296]:
X_char_tr = np.array(X_char_tr)
print(X_char_tr[1])
print(X_char_tr.shape)

[[174   0   0 ...   0   0   0]
 [319   0   0 ...   0   0   0]
 [265   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
(176, 50, 30)


In [297]:
for i in X_char_tr:
    print(i)

[[58  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
[[174   0   0 ...   0   0   0]
 [319   0   0 ...   0   0   0]
 [265   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
[[215   0   0 ...   0   0   0]
 [248   0   0 ...   0   0   0]
 [ 44   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
[[263   0   0 ...   0   0   0]
 [ 44   0   0 ...   0   0   0]
 [285   0   0 ...   0   0   0]
 ...
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]
 [  0   0   0 ...   0   0   0]]
[[10  0  0 ...  0  0  0]
 [20  0  0 ...  0  0  0]
 [39  0  0 ...  0  0  0]
 ...
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]
 [ 0  0  0 ...  0  0  0]]
[[ 44   0   0 ...   0   0   0]
 [285   0   0 ...   0   0   0]
 [300   0   0 ...   0   0   0]
 ...
 [  0   0   0 

In [298]:
x_char = []
for i in words:
    x = []
    for j in i:
        integer_encoded = [char_to_int[char] for char in j]
        x1 = sequence.pad_sequences(np.array([integer_encoded]),maxlen=max_len_char,padding='pre')
        x.append(np.array(x1[0]))
    x_char.append(x[0])
x_char = np.array(x_char)
print(x_char.shape)

(176, 30)


In [299]:
print(max_len)

50


In [300]:
xchar = []
for sen in words:
    test1 = []
    for i in sen:
        integer_encode = [char_to_int[char] for char in i]
        integer_encode = sequence.pad_sequences(np.array([integer_encode]),maxlen=max_len_char,padding='pre')
        test1.append(integer_encode[0])
    if len(sen) < max_len:
        for ad in range(max_len - len(sen)):
            test1.append(np.hstack(([np.zeros(30)])))
    xchar.append(test1)
xchar = np.array(xchar)
print(xchar.shape)

(176, 50, 30)


In [301]:
print(X_word_train.shape)

(176, 50)


In [302]:
def prepare_sequence_target(input_label):
    idxs = [check[w] for w in input_label]
    return idxs

In [384]:
print(len(target_c))

176


In [399]:
y_tr = [prepare_sequence_target(s) for s in target_c]
n_check = len(check)

In [400]:
y_tr = sequence.pad_sequences(maxlen=max_len, sequences=y_tr, padding='pre')

In [401]:
y_tr = [to_categorical(i, num_classes=n_check) for i in y_tr]
y_tr = np.array(y_tr)

In [402]:
print(y_tr.shape)

(176, 50, 3)


In [469]:
word_in = keras.layers.Input(shape=(max_len,), name='word_input_')
word_embeddings = keras.layers.Embedding(input_dim=vecter.shape[0],
                            output_dim=vecter.shape[1],
                            weights = [vecter],input_length=max_len,
                            mask_zero=False,
                            name='word_embedding', trainable=True)(word_in)
char_in = keras.layers.Input(shape=(max_len, max_len_char,), name='char_input')
emb_char = keras.layers.TimeDistributed(keras.layers.Embedding(input_dim=n_chars, output_dim=char_embedding_dim, 
                           input_length=max_len_char, mask_zero=False))(char_in)
char_enc = keras.layers.TimeDistributed(keras.layers.Bidirectional(keras.layers.LSTM(units=character_LSTM_unit, 
                                              return_sequences=False, 
                                              recurrent_dropout=lstm_recurrent_dropout)))(emb_char)

In [470]:
all_word_embeddings = keras.layers.concatenate([word_embeddings, char_enc])
all_word_embeddings = keras.layers.SpatialDropout1D(0.3)(all_word_embeddings)

In [471]:
main_lstm = keras.layers.Bidirectional(keras.layers.LSTM(units=main_lstm_unit, return_sequences=True,
                               recurrent_dropout=lstm_recurrent_dropout))(all_word_embeddings)
main_lstm = keras.layers.TimeDistributed(keras.layers.Dense(50, activation="relu"))(main_lstm)


In [475]:
crf = CRF(n_check)  # CRF layer
out = crf(main_lstm)  # output

In [476]:
model = keras.Model([word_in, char_in], out)

model.compile(optimizer="adam", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

Model: "functional_23"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
char_input (InputLayer)         [(None, 50, 30)]     0                                            
__________________________________________________________________________________________________
word_input_ (InputLayer)        [(None, 50)]         0                                            
__________________________________________________________________________________________________
time_distributed_21 (TimeDistri (None, 50, 30, 32)   12928       char_input[0][0]                 
__________________________________________________________________________________________________
word_embedding (Embedding)      (None, 50, 300)      15483000    word_input_[0][0]                
______________________________________________________________________________________

In [477]:
model.fit([X_word_train,np.array(xchar).reshape((len(xchar), max_len, max_len_char))],np.array(y_tr),epochs=50, verbose=1)


Epoch 1/50


AttributeError: in user code:

    E:\ProgramData\Anaconda3\lib\site-packages\tensorflow\python\keras\engine\training.py:806 train_function  *
        return step_function(self, iterator)
    E:\ProgramData\Anaconda3\lib\site-packages\keras_contrib\losses\crf_losses.py:54 crf_loss  *
        crf, idx = y_pred._keras_history[:2]

    AttributeError: 'Tensor' object has no attribute '_keras_history'
